## Предобработка сигналов

In [40]:
import time
# библиотека для расчета времени преобработки

In [41]:
def save_beats(beats,fn,version):
# сохранение ударов в .csv формате

    savedata = np.array(beats, dtype = float)
    outfn = 'data_ecg\\' + fn + '_' + chname + '_v' + version + '.csv'
    print('    Сохранение ', outfn)
    with open(outfn, "wb") as fin:
        np.savetxt(fin, savedata, delimiter = ",", fmt = '%f')
        
    return

На основе изученных исследований были использованы и проверены несколько вариантов предобработки ЭКГ сигналов 
preprocessing + номер предобработки 

---------------------------------------
№ |  fs   | norm | fill  | swt |


 0  | 125 |  yes   |  0   |  no  |


 1  | 360 |  yes   |  0   |  no  |


 2  | 125 |   no    |  0   |  no  |



 3  | 125 |   no    |  m  |  no  |



 4  | 125 |  yes    |  m  |  no  |



 5  | 360 |  yes    |  m  |  no  |



 6  | 125 |  yes    |  0   |  yes |



 7  | 125 |  yes    |  m  |  yes |



 8  | 360 |  yes    |  0   |  no   |



In [42]:
def preprocessing_0(sign, rates):
    
    # 125 частота, нормализация есть, заполнено 0
    
    # старт предобработки сигнала
    start_time = time.time()
    
    out = ecg.ecg(signal = sign, sampling_rate=360, show=False)
    rpeaks = np.zeros_like(sign, dtype='float')
    rpeaks[out['rpeaks']] = 1.0
    remove = np.array([0])
            
    # Разбиение на отдельные удары сердца. Для каждой записи 
    # сердцебиения добавьте классификацию (нормальный / ненормальный).
    beats = np.split(sign, out['rpeaks'])

    for idx, idxval in enumerate(out['rpeaks']):
        firstround = idx == 0
        lastround = idx == len(beats) - 1

        # Пропуск первого и последнего удара
        if (firstround or lastround):
            continue

        # Получение значения классификации, которое 
        # находится на позиции индекса rpeak или рядом с ним.
        fromidx = 0 if idxval < 10 else idxval - 10
        toidx = idxval + 10
        catval = rates[fromidx:toidx].max()
            
        # Пропустить удар, если нет классификации
        if (catval == 0.0):
            remove = np.append(remove, idx)
            continue

        # Нормальный удар классифицируется как 0 и 
        # аномальный как 1 
        catval = catval - 1.0
                
        beats[idx] = np.append(beats[idx], beats[idx+1][:40])
        amp = np.where(beats[idx] == channel[out['rpeaks']][idx])
        amp_samp = amp[0].astype(float)
                
         # Нормализовать показания до диапазона 0-1 для целей МО.
        beats[idx] = (beats[idx] - beats[idx].min()) / beats[idx].ptp()

        # Добавление дополнительных показаний из следующего удара.
        newsize = int((beats[idx].size * 125 / 360) + 0.5)
        beats[idx] = resample(beats[idx], newsize)

        # Пропустить сигнал если превысил длину
        if (beats[idx].size > 187):
            remove = np.append(remove, idx)
            continue

        # Заполнение участка 0 до определенной длины 
        zerocount = 187 - beats[idx].size
        beats[idx] = np.pad(beats[idx], (0, zerocount),  'constant', constant_values=(0.0, 0.0))
                
        # Добавление классификации к данным биений. 
        beats[idx] = np.append(beats[idx], catval)
        amp_samp[0] = int((amp_samp[0] * 125 / 360) + 0.5)
        
        # Добавление положения пика на участке
        beats[idx] = np.append(beats[idx], amp_samp[0])
        if (amp[0].size < 2):
            beats[idx] = np.append(beats[idx], 0.0)
        else:
            amp_samp[1] = int((amp_samp[1] * 125 / 360) + 0.5)
            beats[idx] = np.append(beats[idx], amp_samp[1])
        
    print("--- %s seconds ---" % (time.time() - start_time))          
    return beats,remove

In [43]:
def preprocessing_1(sign,rates):
    
    # 360 частота, нормализация есть, заполнено 0
    
    out = ecg.ecg(signal=sign, sampling_rate=360, show=False)
    rpeaks = np.zeros_like(sign, dtype='float')
    rpeaks[out['rpeaks']] = 1.0
    remove = np.array([0])

    beats = np.split(sign, out['rpeaks'])
    print(len(beats))
    start_time = time.time()
    for idx, idxval in enumerate(out['rpeaks']):
        firstround = idx == 0
        lastround = idx == len(beats) - 1

        if (firstround or lastround):
            continue

        fromidx = 0 if idxval < 10 else idxval - 10
        toidx = idxval + 10
        catval = rates[fromidx:toidx].max()
            
        if (catval == 0.0):
            remove = np.append(remove, idx)
            continue

        catval = catval - 1.0
                
        beats[idx] = np.append(beats[idx], beats[idx+1][:40])
        amp = np.where(beats[idx] == channel[out['rpeaks']][idx])
        amp_samp = amp[0].astype(float)
        
        #fig,(ax1,ax2,ax3) = plt.subplots(3) 
        #ax1.plot(beats[idx])
        #ax1.set_title('Необработанный сигнал ЭКГ с частотой дискретизации 360Гц')
        
        beats[idx] = (beats[idx] - beats[idx].min()) / beats[idx].ptp()

        #ax2.plot(beats[idx])
        #ax2.set_title('Нормализованый сигнал ЭКГ')
        
        if (beats[idx].size > 540):
            remove = np.append(remove, idx)
            continue

        zerocount = 540 - beats[idx].size
        beats[idx] = np.pad(beats[idx], (0, zerocount), 'constant', constant_values=(0.0, 0.0))
        
        #ax3.plot(beats[idx])
        #ax3.set_title('Заполнение сигнала ЭКГ')
        #ax3.set(xlabel='Отсчеты')
        #fig.text(-0.01, 0.5, 'Электрический потенциал, мВ', va='center', rotation='vertical')
        #plt.tight_layout()
        #plt.show()
           
        beats[idx] = np.append(beats[idx], catval)
        
        beats[idx] = np.append(beats[idx], amp_samp[0])
        if (amp[0].size < 2):
            beats[idx] = np.append(beats[idx], 0.0)
        else:
            beats[idx] = np.append(beats[idx], amp_samp[1])
        
    print("--- %s seconds ---" % (time.time() - start_time)) 
    
    return beats,remove

In [44]:
def preprocessing_2(sign,rates):
    
    # 125 частота, нормализация нет, заполнено 0
    
    out = ecg.ecg(signal=sign, sampling_rate=360, show=False)
    rpeaks = np.zeros_like(sign, dtype='float')
    rpeaks[out['rpeaks']] = 1.0
    remove = np.array([0])
            

    beats = np.split(sign, out['rpeaks'])

    for idx, idxval in enumerate(out['rpeaks']):
        firstround = idx == 0
        lastround = idx == len(beats) - 1

        if (firstround or lastround):
            continue


        fromidx = 0 if idxval < 10 else idxval - 10
        toidx = idxval + 10
        catval = rates[fromidx:toidx].max()
            

        if (catval == 0.0):
            remove = np.append(remove, idx)
            continue

        catval = catval - 1.0
                
        beats[idx] = np.append(beats[idx], beats[idx+1][:40])
        amp = np.where(beats[idx] == channel[out['rpeaks']][idx])
        amp_samp = amp[0].astype(float)
                

        newsize = int((beats[idx].size * 125 / 360) + 0.5)
        beats[idx] = resample(beats[idx], newsize)

        if (beats[idx].size > 187):
            remove = np.append(remove, idx)
            continue

        zerocount = 187 - beats[idx].size
        beats[idx] = np.pad(beats[idx], (0, zerocount), 'constant', constant_values=(0.0, 0.0))
                

        beats[idx] = np.append(beats[idx], catval)
        amp_samp[0] = int((amp_samp[0] * 125 / 360) + 0.5)
        beats[idx] = np.append(beats[idx], amp_samp[0])
        if (amp[0].size < 2):
            beats[idx] = np.append(beats[idx], 0.0)
        else:
            amp_samp[1] = int((amp_samp[1] * 125 / 360) + 0.5)
            beats[idx] = np.append(beats[idx], amp_samp[1])
        
               
    return beats,remove

In [45]:
def preprocessing_3(sign,rates):
    
    # 125 частота, нормализация нет, заполнено median

    out = ecg.ecg(signal=sign, sampling_rate=360, show=False)
    rpeaks = np.zeros_like(sign, dtype='float')
    rpeaks[out['rpeaks']] = 1.0
    remove = np.array([0])
            
    beats = np.split(sign, out['rpeaks'])

    for idx, idxval in enumerate(out['rpeaks']):
        firstround = idx == 0
        lastround = idx == len(beats) - 1

        if (firstround or lastround):
            continue

        fromidx = 0 if idxval < 10 else idxval - 10
        toidx = idxval + 10
        catval = rates[fromidx:toidx].max()
            
        if (catval == 0.0):
            remove = np.append(remove, idx)
            continue

        catval = catval - 1.0
                
        beats[idx] = np.append(beats[idx], beats[idx+1][:40])
        amp = np.where(beats[idx] == channel[out['rpeaks']][idx])
        amp_samp = amp[0].astype(float)
                

        newsize = int((beats[idx].size * 125 / 360) + 0.5)
        beats[idx] = resample(beats[idx], newsize)
        
        beats[idx] = (beats[idx] - beats[idx].min()) / beats[idx].ptp()


        if (beats[idx].size > 187):
            remove = np.append(remove, idx)
            continue

        zerocount = 187 - beats[idx].size

        beats[idx] = np.pad(beats[idx], (0, zerocount), 'median')
                
        beats[idx] = np.append(beats[idx], catval)
        
        
        if (amp[0].size < 2):
            beats[idx] = np.append(beats[idx], 0.0)
            amp_samp[0] = int((amp_samp[0] * 125 / 360) + 0.5)
            beats[idx] = np.append(beats[idx], amp_samp[0])
            
        else:
            amp_samp[-1] = int((amp_samp[-1] * 125 / 360) + 0.5)
            amp_samp[-2] = int((amp_samp[-2] * 125 / 360) + 0.5)
            if (amp_samp[-1]-amp_samp[-2] <= 1):
                beats[idx] = np.append(beats[idx], 0.0)
                beats[idx] = np.append(beats[idx], amp_samp[-1])
            else:
                beats[idx] = np.append(beats[idx], amp_samp[-2])
                beats[idx] = np.append(beats[idx], amp_samp[-1])

    return beats,remove

In [46]:
def preprocessing_4(sign,rates):
    
    # 125 частота, нормализация есть, заполнено median 

    out = ecg.ecg(signal=sign, sampling_rate=360, show=False)
    rpeaks = np.zeros_like(sign, dtype='float')
    rpeaks[out['rpeaks']] = 1.0
    remove = np.array([0])
            
    beats = np.split(sign, out['rpeaks'])

    for idx, idxval in enumerate(out['rpeaks']):
        firstround = idx == 0
        lastround = idx == len(beats) - 1

        if (firstround or lastround):
            continue

        fromidx = 0 if idxval < 10 else idxval - 10
        toidx = idxval + 10
        catval = rates[fromidx:toidx].max()
            
        if (catval == 0.0):
            remove = np.append(remove, idx)
            continue

        catval = catval - 1.0
                
        beats[idx] = np.append(beats[idx], beats[idx+1][:40])
        amp = np.where(beats[idx] == channel[out['rpeaks']][idx])
        amp_samp = amp[0].astype(float)
                

        beats[idx] = (beats[idx] - beats[idx].min()) / beats[idx].ptp()

        newsize = int((beats[idx].size * 125 / 360) + 0.5)
        beats[idx] = resample(beats[idx], newsize)

        if (beats[idx].size > 187):
            remove = np.append(remove, idx)
            continue

        zerocount = 187 - beats[idx].size

        beats[idx] = np.pad(beats[idx], (0, zerocount), 'median')
                

        beats[idx] = np.append(beats[idx], catval)
        amp_samp[0] = int((amp_samp[0] * 125 / 360) + 0.5)
        beats[idx] = np.append(beats[idx], amp_samp[0])
        if (amp[0].size < 2):
            beats[idx] = np.append(beats[idx], 0.0)
        else:
            amp_samp[1] = int((amp_samp[1] * 125 / 360) + 0.5)
            beats[idx] = np.append(beats[idx], amp_samp[1])
        
    return beats,remove

In [47]:
def preprocessing_5(sign,rates):
    
    # 360 частота, нормализация есть, заполнено median

    out = ecg.ecg(signal=sign, sampling_rate=360, show=False)
    rpeaks = np.zeros_like(sign, dtype='float')
    rpeaks[out['rpeaks']] = 1.0
    remove = np.array([0])
            
    beats = np.split(sign, out['rpeaks'])

    for idx, idxval in enumerate(out['rpeaks']):
        firstround = idx == 0
        lastround = idx == len(beats) - 1


        if (firstround or lastround):
            continue

        fromidx = 0 if idxval < 10 else idxval - 10
        toidx = idxval + 10
        catval = rates[fromidx:toidx].max()
            

        if (catval == 0.0):
            remove = np.append(remove, idx)
            continue

        catval = catval - 1.0
                

        beats[idx] = np.append(beats[idx], beats[idx+1][:40])
        amp = np.where(beats[idx] == channel[out['rpeaks']][idx])
        amp_samp = amp[0].astype(float)
                

        beats[idx] = (beats[idx] - beats[idx].min()) / beats[idx].ptp()

        if (beats[idx].size > 540):
            remove = np.append(remove, idx)
            continue

        zerocount = 540 - beats[idx].size

        beats[idx] = np.pad(beats[idx], (0, zerocount), 'median')
                
        beats[idx] = np.append(beats[idx], catval)
        beats[idx] = np.append(beats[idx], amp_samp[0])
        if (amp[0].size < 2):
            beats[idx] = np.append(beats[idx], 0.0)
        else:
            beats[idx] = np.append(beats[idx], amp_samp[1])
        
    return beats,remove

In [48]:
def swt_preprocing(ecg):
    
    # предобработка swt преобразованием сигнала с
    # частотой дискретизации 125
    
    filt = np.empty(shape = [192])
    fs = 125
    swt_level = 3
    padding = -1
    
    #fig, (ax1,ax2,ax3) = plt.subplots(3,constrained_layout=True)
    #ax1.plot(ecg)
    #ax1.set_title('Сингнал ЭКГ с частотой дискретизации 125 Гц')
    
    for i in range(1000):
        if (len(ecg) + i) % 2 ** swt_level == 0:
            padding = i
            break

    if padding > 0:
        ecg = np.pad(ecg, (0, padding), "edge")
    elif padding == -1:
        print("Padding greater than 1000 required\n")

    #ax2.plot(ecg)
    #ax2.set_title(ylabel = 'Удар ' + str(beatid) + " тип " + str(anno))
    
    swt_ecg = pywt.swt(ecg, "db3", level=swt_level)
    swt_ecg = np.array(swt_ecg)
    swt_ecg = swt_ecg[0, 1, :]
    
    #ax2.plot(swt_ecg)
    #ax2.set_title('Сигнал ЭКГ после SWT предобработки')
    
    squared = swt_ecg * swt_ecg

    f1 = 0.01 / fs
    f2 = 10 / fs

    b, a = scipy.signal.butter(3, [f1 * 2, f2 * 2], btype="bandpass")
    filtered_squared = scipy.signal.lfilter(b, a, squared)
    
    #ax3.plot(filtered_squared)
    #ax3.set_title('Сигнал ЭКГ после полосовой фильтрации и возведения в квадрат')
    #plt.figure(figsize=(10, 10))
    #plt.tight_layout()
    #plt.show()
    
    return filtered_squared

In [49]:
def preprocessing_6(sign,rates):
    
    # 125 частота, нормализация есть, заполнено 0
    
    out = ecg.ecg(signal=sign, sampling_rate=360, show=False)
    rpeaks = np.zeros_like(sign, dtype='float')
    rpeaks[out['rpeaks']] = 1.0
    remove = np.array([0])
            

    beats = np.split(sign, out['rpeaks'])

    for idx, idxval in enumerate(out['rpeaks']):
        firstround = idx == 0
        lastround = idx == len(beats) - 1

    
        if (firstround or lastround):
            continue

        fromidx = 0 if idxval < 10 else idxval - 10
        toidx = idxval + 10
        catval = rates[fromidx:toidx].max()
            
        if (catval == 0.0):
            remove = np.append(remove, idx)
            continue

        catval = catval - 1.0
                
        beats[idx] = np.append(beats[idx], beats[idx+1][:40])
        amp = np.where(beats[idx] == channel[out['rpeaks']][idx])
        amp_samp = amp[0].astype(float)
                
        beats[idx] = (beats[idx] - beats[idx].min()) / beats[idx].ptp()


        newsize = int((beats[idx].size * 125 / 360) + 0.5)
        beats[idx] = resample(beats[idx], newsize)


        if (beats[idx].size > 192):
            remove = np.append(remove, idx)
            continue

        beats[idx] = swt_preprocing(beats[idx])
        
        zerocount = 192 - beats[idx].size
        beats[idx] = np.pad(beats[idx], (0, zerocount), 'constant', constant_values=(0.0, 0.0))
                
          
        beats[idx] = np.append(beats[idx], catval)
        amp_samp[0] = int((amp_samp[0] * 125 / 360) + 0.5)
        
        beats[idx] = np.append(beats[idx], amp_samp[0])
        if (amp[0].size < 2):
            beats[idx] = np.append(beats[idx], 0.0)
        else:
            amp_samp[1] = int((amp_samp[1] * 125 / 360) + 0.5)
            beats[idx] = np.append(beats[idx], amp_samp[1])
                       
    return beats,remove

In [50]:
def preprocessing_7(sign,rates):
    
    # 125 частота, нормализация есть, заполнено mediana
    
    out = ecg.ecg(signal=sign, sampling_rate=360, show=False)
    rpeaks = np.zeros_like(sign, dtype='float')
    rpeaks[out['rpeaks']] = 1.0
    remove = np.array([0])
            
    beats = np.split(sign, out['rpeaks'])

    for idx, idxval in enumerate(out['rpeaks']):
        firstround = idx == 0
        lastround = idx == len(beats) - 1

    
        if (firstround or lastround):
            continue

        fromidx = 0 if idxval < 10 else idxval - 10
        toidx = idxval + 10
        catval = rates[fromidx:toidx].max()
            

        if (catval == 0.0):
            remove = np.append(remove, idx)
            continue
       
        catval = catval - 1.0
                
        beats[idx] = np.append(beats[idx], beats[idx+1][:40])
        amp = np.where(beats[idx] == channel[out['rpeaks']][idx])
        amp_samp = amp[0].astype(float)

        newsize = int((beats[idx].size * 125 / 360) + 0.5)
        beats[idx] = resample(beats[idx], newsize)

        if (beats[idx].size > 192):
            remove = np.append(remove, idx)
            continue

        beats[idx] = swt_preprocing(beats[idx])
        
        zerocount = 192 - beats[idx].size
        beats[idx] = np.pad(beats[idx], (0, zerocount), 'median')
                
          
        beats[idx] = np.append(beats[idx], catval)
        amp_samp[0] = int((amp_samp[0] * 125 / 360) + 0.5)
        
        beats[idx] = np.append(beats[idx], amp_samp[0])
        if (amp[0].size < 2):
            beats[idx] = np.append(beats[idx], 0.0)
        else:
            amp_samp[1] = int((amp_samp[1] * 125 / 360) + 0.5)
            beats[idx] = np.append(beats[idx], amp_samp[1])
                    
    return beats,remove

In [51]:
def preprocessing_8(sign,rates):
    
    # 360 частота, нормализация есть, заполнено 0
    
    out = ecg.ecg(signal=sign, sampling_rate=360, show=False)
    rpeaks = np.zeros_like(sign, dtype='float')
    rpeaks[out['rpeaks']] = 1.0
    remove = np.array([0])

    beats = np.split(sign, out['rpeaks'])

    for idx, idxval in enumerate(out['rpeaks']):
        firstround = idx == 0
        lastround = idx == len(beats) - 1

        if (firstround or lastround):
            continue

        fromidx = 0 if idxval < 10 else idxval - 10
        toidx = idxval + 10
        catval = rates[fromidx:toidx].max()

        if (catval == 0.0):
            remove = np.append(remove, idx)
            continue

        catval = catval - 1.0
                
        beats[idx] = np.append(beats[idx], beats[idx+1][:40])
        amp = np.where(beats[idx] == channel[out['rpeaks']][idx])
        amp_samp = amp[0].astype(float)
        
         
        if (beats[idx].size > 540):
            remove = np.append(remove, idx)
            continue

           
        zerocount = 540 - beats[idx].size
        beats[idx] = np.pad(beats[idx], (0, zerocount), 'constant', constant_values=(0.0, 0.0))

        
        beats[idx] = np.append(beats[idx], catval)
  
        if (amp[0].size < 2):
            beats[idx] = np.append(beats[idx], 0.0)
            beats[idx] = np.append(beats[idx], amp_samp[0])
            
        else:
            if (amp_samp[-1] - amp_samp[-2] <= 1):
                beats[idx] = np.append(beats[idx], 0.0)
                beats[idx] = np.append(beats[idx], amp_samp[-1])
            else:
                beats[idx] = np.append(beats[idx], amp_samp[-2])
                beats[idx] = np.append(beats[idx], amp_samp[-1])
               
    return beats,remove

In [52]:
def change_annotation(annotation, data):
    
    # Сгенерирование классификации на основе анотаций
    # 0.0 = неопределенный
    # 1.0 = нормальный
    # 2.0 = аномальный
    
    cat = np.array(annotation.symbol)
    rate = np.zeros_like(cat, dtype='float')
    
    for catid, catval in enumerate(cat):
        if (catval == 'N'):
            rate[catid] = 1.0 # нормальный
            
        elif (catval in realbeats):
            rate[catid] = 2.0 # аномальный
            
    rates = np.zeros_like(data[0], dtype='float')
    rates[annotation.sample] = rate
    
    return rates

In [53]:
pip install biosppy

Note: you may need to restart the kernel to use updated packages.


In [54]:
# библиотека для работы с каталогами и 
import os
import wfdb as wf
import numpy as np
import matplotlib.pyplot as plt
# Набор инструментов обработки сигналов
from scipy.signal import resample
from datasets import mitdb as dm
# модуль предоставляет методы 
# обработки сигналов электрокардиографии (ЭКГ)
from biosppy.signals import ecg
# программное обеспечение для вейвлет-преобразования
import pywt
import scipy
from scipy import signal

In [55]:
records = dm.get_records()
print(records)
print('Всего файлов: ', len(records))

# Вместо использования аннотаций для поиска ударов используется
# обнаружение R-пика. Причина этого в том, что ту же логику 
# можно использовать для анализа новых и не аннотированных 
# данных ЭКГ. Aннотации здесь только для того, 
# чтобы классифицировать ритм как нормальный или ненормальный, 
# а также для обучения модели.
realbeats = ['N','L','R','B','A','a','J','S','V','r',
             'F','e','j','n','E','/','f','Q','?']

[]
Всего файлов:  0


In [35]:
def preproc_and_save(channel, rates, fn, num):
    
    # channel - запись сигнала
    # rates - последовательность с аннотациями
    # fn - номер файла
    # num - номер предобработки
    
    a = 'preprocessing_' + num
    if num == '0':
        beats, beatsremove = preprocessing_0( channel, rates )
    elif num == '1':
        beats, beatsremove = preprocessing_1( channel, rates )
    elif num == '2':
        beats, beatsremove = preprocessing_2( channel, rates )
    elif num == '3':
        beats, beatsremove = preprocessing_3( channel, rates )
    elif num == '4':
        beats, beatsremove = preprocessing_4( channel, rates )
    elif num == '5':
        beats, beatsremove = preprocessing_5( channel, rates )
    elif num == '6':
        beats, beatsremove = preprocessing_6( channel, rates )
    elif num == '7':
        beats, beatsremove = preprocessing_7( channel, rates )
    else:
        beats, beatsremove = preprocessing_8( channel, rates )
        
    beatsremove = np.append( beatsremove, len(beats) - 1 )
    beats = np.delete( beats, beatsremove )
    save_beats( list(beats[:]), fn, num )
    
    return

In [36]:
import timeit
for path in records:
    pathpts = path.split('\\')
    fn = pathpts[-1]
    print('Загрузка файла:', path)

    # Чтение информации
    record = wf.rdsamp(path)
    annotation = wf.rdann(path, 'atr')
    print('    Количество загруженных аннотаций:', len(annotation.num))
    
    # Получение значений ЭКГ из файла
    data = record[0].transpose()
    
    rates = change_annotation(annotation,data)
    
    if fn != '233' or fn != '234':
        
        for channelid, channel in enumerate(data):
            chname = record[1].get('sig_name')[channelid]
            print('    Тип ЭКГ канала:', chname)
            
            preproc_and_save(channel,rates,fn,'0')
            preproc_and_save(channel,rates,fn,'1')
            preproc_and_save(channel,rates,fn,'2')
            preproc_and_save(channel,rates,fn,'3')
            preproc_and_save(channel,rates,fn,'4')
            preproc_and_save(channel,rates,fn,'5')
            preproc_and_save(channel,rates,fn,'6')
            preproc_and_save(channel,rates,fn,'7')
            preproc_and_save(channel,rates,fn,'8')